In [1]:

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))


/var/folders/0p/k705g0x102v0x_kgxydj12kc0000gn/T/ipykernel_20731/2853790449.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import os
import glob
import pandas as pd
import natsort

In [3]:
col_list = ['YMD', '3시간기온', '12시간강수량', '12시간신적설', '강수형태', '강수확률', '습도', '일최고기온', '일최저기온', '파고', '풍속', '풍향', '하늘상태']

In [4]:
from datetime import date, timedelta, datetime
def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates

In [6]:
# 2012년 '
folder_2012 = os.listdir(f'/Users/simcheol-u/Desktop/빅_데이터_콘테스트/data/예보_데이터/2015')
folder_2012 = natsort.natsorted(folder_2012)



for area in folder_2012:
    print(area)
    file_list = os.listdir(f'/Users/simcheol-u/Desktop/빅_데이터_콘테스트/data/예보_데이터/2015/{area}')
    file_list = natsort.natsorted(file_list)

    # 빈 일별 데이터 프레임 생성
    dates = pd.DataFrame(columns=col_list)
    dates['YMD'] = date_range('2015-01-01', '2015-12-31')
    
    data = pd.DataFrame()
    
    counts = 1

    for file in file_list:
        print(file)
        file = pd.read_csv(f'/Users/simcheol-u/Desktop/빅_데이터_콘테스트/data/예보_데이터/2015/{area}/{file}')

        # 중복 데이터 제거
        if '  format: day' in file[' format: day'].values:
            x = file[file[' format: day'] == '  format: day'].index[0]
            file = file.iloc[:x, :]
        
        if counts == 9:
            counts += 1
            continue
        
        # 일자 추가
        days = file.loc[:, ' format: day'].str.replace(' ', '')                

        month_index = file[file[' format: day'].str.contains('Star')].index
        month = 1
        i = 0
        
        for n in range(len(file[' format: day'])):
            if days[n].find('Start') != -1:
                month = file.loc[month_index[i], ' format: day'].replace(' Start : ', '')[4:6]
                i += 1

            file.loc[n, 'YMD'] = f'2015-{month}-{days[n]}'
        
        # 'Start%' drop
        start_index = file[file[' format: day'].str.contains('Start')].index
        file.drop(index=start_index, inplace=True)

        # object -> numeric
        for x in range(len(file.columns)-1):
            file.iloc[:, x] = pd.to_numeric(file.iloc[:, x])

        # 일단위 예측 범위 설정
        file = file[file['hour'] == 200]
        file = file[file['forecast'] <= 22]
        file.reset_index(drop=True, inplace=True)
        
        # datetime type으로 변환 - > str
        file['YMD'] = pd.to_datetime(file['YMD'])
        file['YMD'] = file['YMD'].astype(str)
        
        
        # 일별 평균
        mean_value = file.groupby(['YMD'], as_index=False).mean()
        
        
        for indexs in range(len(mean_value['YMD'])):
            target_index = dates[mean_value.loc[indexs, 'YMD'] == dates['YMD']].index
            try:
                dates.iloc[target_index, counts] = mean_value.iloc[indexs, 4]
            except:
                continue
        counts += 1
        
        if counts == 13:
            data = pd.concat([data, dates], ignore_index=False)
            print(data)
            counts = 1

    data['area'] = area
    data.to_csv(path_or_buf=f'/Users/simcheol-u/Desktop/빅_데이터_콘테스트/data/예보_모음/2015_{area}.csv')

강원
갈말읍_3시간기온_201501_201512.csv
갈말읍_6시간강수량_201501_201512.csv
갈말읍_6시간적설_201501_201512.csv
갈말읍_강수형태_201501_201512.csv
갈말읍_강수확률_201501_201512.csv
갈말읍_습도_201501_201512.csv
갈말읍_일최고기온_201501_201512.csv
갈말읍_일최저기온_201501_201512.csv
갈말읍_파고_201501_201512.csv
갈말읍_풍속_201501_201512.csv
갈말읍_풍향_201501_201512.csv
갈말읍_하늘상태_201501_201512.csv
            YMD     3시간기온   12시간강수량   12시간신적설      강수형태       강수확률         습도  \
0    2015-01-01      -9.0       0.0       0.0       0.0   8.571429  52.857143   
1    2015-01-02      -9.0       0.0       0.0       0.0        0.0  57.142857   
2    2015-01-03 -2.714286  0.333333  0.333333  0.428571       30.0  72.142857   
3    2015-01-04 -0.571429       0.0       0.0       0.0   7.142857       85.0   
4    2015-01-05  0.428571  3.333333       2.0  0.857143  41.428571  81.428571   
..          ...       ...       ...       ...       ...        ...        ...   
360  2015-12-27 -8.857143       0.0       0.0       0.0        0.0       55.0   
361  2015-12-28 -7.714286  

송정동_6시간강수량_201501_201512.csv
송정동_6시간적설_201501_201512.csv
송정동_강수형태_201501_201512.csv
송정동_강수확률_201501_201512.csv
송정동_습도_201501_201512.csv
송정동_일최고기온_201501_201512.csv
송정동_일최저기온_201501_201512.csv
송정동_파고_201501_201512.csv
송정동_풍속_201501_201512.csv
송정동_풍향_201501_201512.csv
송정동_하늘상태_201501_201512.csv
            YMD     3시간기온   12시간강수량   12시간신적설      강수형태       강수확률         습도  \
0    2015-01-01      -9.0       0.0       0.0       0.0   8.571429  52.857143   
1    2015-01-02      -9.0       0.0       0.0       0.0        0.0  57.142857   
2    2015-01-03 -2.714286  0.333333  0.333333  0.428571       30.0  72.142857   
3    2015-01-04 -0.571429       0.0       0.0       0.0   7.142857       85.0   
4    2015-01-05  0.428571  3.333333       2.0  0.857143  41.428571  81.428571   
..          ...       ...       ...       ...       ...        ...        ...   
360  2015-12-27 -1.857143  0.333333  0.333333  0.857143  21.428571  58.571429   
361  2015-12-28       0.0       0.0       0.0       0.0   

인제읍_강수확률_201501_201512.csv
인제읍_습도_201501_201512.csv
인제읍_일최고기온_201501_201512.csv
인제읍_일최저기온_201501_201512.csv
인제읍_파고_201501_201512.csv
인제읍_풍속_201501_201512.csv
인제읍_풍향_201501_201512.csv
인제읍_하늘상태_201501_201512.csv
            YMD     3시간기온   12시간강수량   12시간신적설      강수형태       강수확률         습도  \
0    2015-01-01      -9.0       0.0       0.0       0.0   8.571429  52.857143   
1    2015-01-02      -9.0       0.0       0.0       0.0        0.0  57.142857   
2    2015-01-03 -2.714286  0.333333  0.333333  0.428571       30.0  72.142857   
3    2015-01-04 -0.571429       0.0       0.0       0.0   7.142857       85.0   
4    2015-01-05  0.428571  3.333333       2.0  0.857143  41.428571  81.428571   
..          ...       ...       ...       ...       ...        ...        ...   
360  2015-12-27 -6.571429       0.0       0.0       0.0        0.0  52.142857   
361  2015-12-28 -7.142857       0.0       0.0       0.0   1.428571       55.0   
362  2015-12-29 -4.428571       0.0       0.0       0.0  14.2

홍천읍_일최저기온_201501_201512.csv
홍천읍_파고_201501_201512.csv
홍천읍_풍속_201501_201512.csv
홍천읍_풍향_201501_201512.csv
홍천읍_하늘상태_201501_201512.csv
            YMD     3시간기온   12시간강수량   12시간신적설      강수형태       강수확률         습도  \
0    2015-01-01      -9.0       0.0       0.0       0.0   8.571429  52.857143   
1    2015-01-02      -9.0       0.0       0.0       0.0        0.0  57.142857   
2    2015-01-03 -2.714286  0.333333  0.333333  0.428571       30.0  72.142857   
3    2015-01-04 -0.571429       0.0       0.0       0.0   7.142857       85.0   
4    2015-01-05  0.428571  3.333333       2.0  0.857143  41.428571  81.428571   
..          ...       ...       ...       ...       ...        ...        ...   
360  2015-12-27 -6.142857       0.0       0.0       0.0        0.0  46.428571   
361  2015-12-28 -6.714286       0.0       0.0       0.0   1.428571  57.857143   
362  2015-12-29 -3.142857       0.0       0.0       0.0  14.285714  71.428571   
363  2015-12-30 -1.714286       2.0       2.0  1.285714  45.

생연1동_풍향_201501_201512.csv
생연1동_하늘상태_201501_201512.csv
            YMD     3시간기온   12시간강수량   12시간신적설      강수형태       강수확률         습도  \
0    2015-01-01      -9.3       0.0       0.0       0.0   8.571429  60.857143   
1    2015-01-02 -8.414286       0.0       0.0       0.0   2.857143  61.142857   
2    2015-01-03 -0.914286       0.0       0.0       0.0  22.428571  73.714286   
3    2015-01-04       0.2       0.0       0.0       0.0   8.571429  63.571429   
4    2015-01-05       1.5  3.333333  0.666667  0.714286  44.428571  71.428571   
..          ...       ...       ...       ...       ...        ...        ...   
360  2015-12-27 -7.571429       0.0       0.0       0.0        0.0  45.714286   
361  2015-12-28 -6.142857       0.0       0.0       0.0   1.428571  55.714286   
362  2015-12-29 -3.142857       0.0       0.0       0.0  18.571429  65.714286   
363  2015-12-30 -0.285714  3.333333  3.333333  1.285714  42.857143  79.285714   
364  2015-12-31 -1.428571       0.0       0.0       0.0

...

            YMD     3시간기온   12시간강수량   12시간신적설      강수형태       강수확률         습도  \
0    2015-01-01 -3.942857  0.666667       2.0  1.285714  37.428571  74.142857   
1    2015-01-02 -3.585714  0.333333  0.333333  0.428571  13.428571  78.428571   
2    2015-01-03  0.442857       0.0       0.0       0.0       19.0  76.571429   
3    2015-01-04  3.957143       0.0       0.0       0.0   5.857143  75.285714   
4    2015-01-05  5.471429  6.666667       0.0  0.571429  51.857143  84.571429   
..          ...       ...       ...       ...       ...        ...        ...   
360  2015-12-27 -4.285714       0.0       0.0       0.0   2.857143  63.571429   
361  2015-12-28 -5.285714       0.0       0.0       0.0  17.142857  67.857143   
362  2015-12-29 -3.714286       0.0       0.0       0.0  14.285714  72.857143   
363  2015-12-30  0.142857  3.333333  3.333333  0.857143  37.142857  72.142857   
364  2015-12-31 -2.571429       0.0       0.0       0.0   2.857143  72.857143   

        일최고기온 일최저기온   파고   

한경면_6시간강수량_201501_201512.csv
한경면_6시간적설_201501_201512.csv
한경면_강수형태_201501_201512.csv
한경면_강수확률_201501_201512.csv
한경면_습도_201501_201512.csv
한경면_일최고기온_201501_201512.csv
한경면_일최저기온_201501_201512.csv
한경면_파고_201501_201512.csv
한경면_풍속_201501_201512.csv
한경면_풍향_201501_201512.csv
한경면_하늘상태_201501_201512.csv
            YMD      3시간기온   12시간강수량 12시간신적설      강수형태       강수확률         습도  \
0    2015-01-01   2.428571  0.666667     0.0  1.714286  42.857143  58.571429   
1    2015-01-02   4.285714  0.333333     0.0  0.285714       20.0  61.428571   
2    2015-01-03   8.428571       0.0     0.0       0.0  14.285714  57.142857   
3    2015-01-04  10.285714       0.0     0.0       0.0   7.142857       60.0   
4    2015-01-05  13.714286       5.0     0.0  0.428571  44.285714       80.0   
..          ...        ...       ...     ...       ...        ...        ...   
360  2015-12-27   4.628571       0.0     0.0       0.0       20.0       61.0   
361  2015-12-28   5.428571       0.0     0.0       0.0       20.0 

부여읍_강수형태_201501_201512.csv
부여읍_강수확률_201501_201512.csv
부여읍_습도_201501_201512.csv
부여읍_일최고기온_201501_201512.csv
부여읍_일최저기온_201501_201512.csv
부여읍_파고_201501_201512.csv
부여읍_풍속_201501_201512.csv
부여읍_풍향_201501_201512.csv
부여읍_하늘상태_201501_201512.csv
            YMD     3시간기온   12시간강수량   12시간신적설      강수형태       강수확률         습도  \
0    2015-01-01 -6.142857       0.0       0.0  0.428571  21.428571  68.571429   
1    2015-01-02 -7.571429  0.333333  0.333333  0.428571       10.0  77.857143   
2    2015-01-03 -2.142857       0.0       0.0       0.0  18.571429  81.428571   
3    2015-01-04       0.0       0.0       0.0       0.0   7.142857  80.714286   
4    2015-01-05  4.142857       5.0       0.0  0.428571  48.571429  80.714286   
..          ...       ...       ...       ...       ...        ...        ...   
360  2015-12-27 -3.285714       0.0       0.0       0.0   1.428571  64.285714   
361  2015-12-28 -3.428571       0.0       0.0       0.0       10.0       74.0   
362  2015-12-29 -2.142857       0.

교현.안림동_일최저기온_201501_201512.csv
교현.안림동_파고_201501_201512.csv
교현.안림동_풍속_201501_201512.csv
교현.안림동_풍향_201501_201512.csv
교현.안림동_하늘상태_201501_201512.csv
            YMD     3시간기온   12시간강수량   12시간신적설      강수형태       강수확률         습도  \
0    2015-01-01 -6.042857       0.0       0.0  0.428571       18.0  60.142857   
1    2015-01-02 -5.357143       0.0       0.0       0.0   2.714286  60.428571   
2    2015-01-03 -1.357143       0.0       0.0       0.0       18.0  74.571429   
3    2015-01-04  1.742857       0.0       0.0       0.0   6.571429  74.428571   
4    2015-01-05  3.928571       5.0       0.0  0.428571  44.571429  74.142857   
..          ...       ...       ...       ...       ...        ...        ...   
360  2015-12-27      -5.0       0.0       0.0       0.0        0.0  53.571429   
361  2015-12-28 -5.142857       0.0       0.0       0.0   8.571429  59.285714   
362  2015-12-29 -2.714286       0.0       0.0       0.0  11.428571  69.571429   
363  2015-12-30  0.142857  3.333333  3.333333

추풍령면_풍향_201501_201512.csv
추풍령면_하늘상태_201501_201512.csv
            YMD     3시간기온   12시간강수량   12시간신적설      강수형태       강수확률         습도  \
0    2015-01-01 -6.042857       0.0       0.0  0.428571       18.0  60.142857   
1    2015-01-02 -5.357143       0.0       0.0       0.0   2.714286  60.428571   
2    2015-01-03 -1.357143       0.0       0.0       0.0       18.0  74.571429   
3    2015-01-04  1.742857       0.0       0.0       0.0   6.571429  74.428571   
4    2015-01-05  3.928571       5.0       0.0  0.428571  44.571429  74.142857   
..          ...       ...       ...       ...       ...        ...        ...   
360  2015-12-27 -3.142857       0.0       0.0       0.0        0.0  48.571429   
361  2015-12-28 -3.285714       0.0       0.0       0.0   8.571429  52.857143   
362  2015-12-29 -2.142857       0.0       0.0       0.0  11.428571  54.285714   
363  2015-12-30  0.428571  3.333333  3.333333  0.857143  35.714286       70.0   
364  2015-12-31 -0.142857       0.0       0.0       0.0

In [ ]:
# 2012년 - /Volumes/GoogleDrive/내 드라이브/Colab Notebooks/예보데이터/데이터셋/2015
foder_2012 = os.listdir(f'/Users/ohbs/Desktop/날씨 빅데이터 콘테스트/데이터셋/2015/')
foder_2012 = natsort.natsorted(foder_2012)

for area in foder_2012:    
    file_list = os.listdir(f'/Users/ohbs/Desktop/날씨 빅데이터 콘테스트/데이터셋/2015/{area}')
    file_list = natsort.natsorted(file_list)

    for file in file_list:
        print(file)
        file = pd.read_csv(f'/Users/ohbs/Desktop/날씨 빅데이터 콘테스트/데이터셋/2015/{area}/{file}')

        # 중복 데이터 제거
        if '  format: day' in file[' format: day'].values:
            x = file[file[' format: day'] == '  format: day'].index[0]
        file = file.iloc[:x, :]

        # 일자 추가
        days = file.loc[:, ' format: day'].str.replace(' ', '')    
        month_index = file[file[' format: day'].str.contains('Start')].index
        month = 1
        i = 0
        
        for n in range(len(file[' format: day'])):
            if days[n].find('Start') != -1:
                month = file.loc[month_index[i], ' format: day'].replace(' Start : ', '')[4:6]
                i += 1
                
            file.loc[n, 'YMD'] = f'2012-{month}-{days[n]}'

        # 'Start%' drop
        start_index = file[file[' format: day'].str.contains('Start')].index
        file.drop(index=start_index, inplace=True)

        # object -> numeric
        for x in range(len(file.columns)-1):
            file.iloc[:, x] = pd.to_numeric(file.iloc[:, x])

        # 일단위 예측 범위 설정
        file = file[file['hour'] == 200]
        file = file[file['forecast'] <= 22]
        file.reset_index(drop=True, inplace=True)
        
        

밑에 코드는 무시해주세요!! 실행도 놉! 

In [ ]:
# 일별 평균
mean_value = file.groupby(['YMD'], as_index=False).mean()
print(mean_value.iloc[:, 4])

# 빈 일별 데이터 프레임 생성
dates = pd.DataFrame(columns=col_list)
dates['YMD'] = pd.date_range('2015-01-01', '2015-12-31')

In [ ]:
""" 
# 예보 데이터 파일 > 년도(2012, 2013, 2014, 2015) > 지역(강원, ...충북) > 파일 순으로 진행

forders_top = os.listdir('/Users/ohbs/Desktop/날씨 빅데이터 콘테스트/데이터셋/')
forders_top = natsort.natsorted(forders_top)

# 년도 디렉토리 선택
for year_dir in forders_top:

# 윤년 확인
if (int2015 % 4) == 0:
    year_type = leap_year
else:
    year_type = year
"""

In [ ]:
"""
def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates
"""

In [ ]:

# 내꺼
forders_sec = os.listdir(f'/Users/ohbs/Desktop/날씨 빅데이터 콘테스트/데이터셋/2015')
forders_sec = natsort.natsorted(forders_sec)


# 지역 디렉토리 선택
for area_dir in forders_sec:
    file_list = os.listdir(f'/Users/ohbs/Desktop/날씨 빅데이터 콘테스트/데이터셋/2015/{area_dir}')
    file_list = natsort.natsorted(file_list)
    
    ### (수정필요) / 201%-01-01 ~ 201%-12-31 기간의 일별 정보를 담고 있는 빈 데이터 프레임 생성 해야함
    width_frame = pd.DataFrame()
    data = pd.DataFrame()

    # 컬럼 이름과 데이터 type을 구분 하기 위한 count
    counts = 0
    # 파일 선택
    for files in file_list:
        # 파일 업로드
        file = pd.read_csv(f'/Users/ohbs/Desktop/날씨 빅데이터 콘테스트/데이터셋/2015/{area_dir}/{files}', encoding='euc-kr')
        
        ### (수정필요) object -> numeric / 기존에 drop 했던 해당 요소를 이용하여 '월'을 카운트 해야함
        file = file.drop(file[file.iloc[:, 0].str.contains('Start')].index)
        file = file.drop(file[file.iloc[:, 0].str.contains('format')].index)
        for x in range(len(file.columns)):
            file.iloc[:, x] = pd.to_numeric(file.iloc[:, x])

        ### (수정필요) 일단위 예측 범위 설정
        file = file[file['hour'] == 200]
        file = file[file['forecast'] <= 22]
        file.reset_index(drop=True, inplace=True)
        
        ### (수정필요) / 아마 싹 바꿔야 할듯..
        # '일'을 선택할 인덱스 시작과 끝
        start = 0
        end = 0
        print(file)
        if counts in type_A:
            # 일단위 예측 범위 설정 후 '일'별 forcast 수가 7개이기 때문에  7칸씩 선택
            day_index = 7
            month = 1
            for month_days in year:
                for days in range(1, month_days+1):
                    end += day_index
                    file.loc[start:end, 'YYYY.MM.DD'] = f'2015-{month}-{days}'
                    start = end
                month += 1
            print(file)
            # 월별 평균
            mean_value = file.groupby(['YYYY.MM.DD'], as_index=False).mean()
            print(mean_value)
            # 열로 붙임
            width_frame = pd.concat([width_frame, mean_value['YYYY.MM.DD']], axis=1)
            width_frame.rename(columns={('YYYY.MM.DD'): (col_list[counts])}, inplace=True)
            counts += 1
                
        elif counts in type_B:
            day_index = 4
            month = 1
            for month_days in year:
                for days in range(1, month_days+1):
                    end += day_index
                    file.loc[start:end, 'YYYY.MM.DD'] = f'2015-{month}-{days}'
                    start = end
                month += 1
            mean_value = file.groupby(['YYYY.MM.DD'], as_index=False).mean()
            width_frame = pd.concat([width_frame, mean_value['YYYY.MM.DD']], axis=1)
            width_frame.rename(columns={('YYYY.MM.DD'): (col_list[counts])}, inplace=True)
            counts += 1
            
        elif counts in type_C:
            day_index = 2
            month = 1
            for month_days in year:
                for days in range(1, month_days+1):
                    end += day_index
                    file.loc[start:end, 'YYYY.MM.DD'] = f'2015-{month}-{days}'
                    start = end
                month += 1
            mean_value = file.groupby(['YYYY.MM.DD'], as_index=False).mean()
            width_frame = pd.concat([width_frame, mean_value['YYYY.MM.DD']], axis=1)
            width_frame.rename(columns={('YYYY.MM.DD'): (col_list[counts])}, inplace=True)
            counts += 1
            
        elif counts in type_D:
            counts += 1
        
        # 총 13개의 colum이 있기 때문에 counts가 12가 되면 빈데이터 dataframe에 추가 후 해당 디렉토리 안에 있는 파일이 끝날 때까지 반복
        if counts == 13:
            data = pd.concat([data, width_frame], ignore_index=False)
            counts = 0
    
    # 한 지역이 끝나면 해당 디렉토리에 추가
    data['area'] = area_dir
    data.to_csv(path_or_buf='/Users/ohbs/Desktop/날씨 빅데이터 콘테스트/New_데이터셋/'+f'2015_{area_dir}.csv')

취합 순서
1. 해당 년도의 각 지역별로 취합 (2012 : 강원, 2012 : 경기...) /  (2013 : 강원, 2012 : 경기...) ..
2. 각 년도 별로 지역 취합 (2012, 2013 ...)
3. 년도 취합

문제
1. forecast의 수가 다르다.

    문제 이유: 각 feature들이 파일형태로 찢어져 있기 때문에 파일을 열로 붙여줘야 함. 원 형태로는 붙일 수가 없음.
    
    ex)
    
    file = file[file['hour'] == 200]

    file = file[file['forecast'] <= 22]
    
    이후 일별로 '3시간 기온'은 forecast가 7개 남고, '12시간 강수량'은 4개가 남음 -> 파일마다 row수가 다르게 남는다.
    
때문에 4개의 type으로 구분함

['3시간기온', '12시간강수량', '12시간신적설', '강수형태', '강수확률', '습도', '일최고기온', '일최저기온', '파고', '풍속', '풍향', '하늘상태'] 순으로 forecast 수는

type_A = [0, 3, 4, 5, 9, 10, 11] / 7개

type_B = [1, 2] / 4개

type_C = [6, 7] / 2개

type_D = [8] / '파고' 인데 해당 되지 않는 지역이 있어서 예외 처리 해줘야한다.

해결방안 : 해당 파일의 일별 forecast를 각각 평균을 내주어 1일에 1row로 만들어 준 후, 년월일 정보를 가지고 있는 빈 데이터 프레임에 concat

2. 평균을 내어 1일에 1row로 만들기 위해서는 년월일 정보가 있어야 한다.

년: 해당 디렉토리 이름

월: 월이 바뀔 때마다 생성 되어있는 'Start'정보를 이용![%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202022-07-21%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%208.20.23.png](attachment:%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202022-07-21%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%208.20.23.png)
(정확히 확인 하지는 않았으나 'start' -> 'format'로 변경 되는 것과같이  년별로 표기가 상이한 부분이 있음)

일: 'format: day'

(년별로 datatype이 다르기 때문에 인덱싱 하는 것이 까다로움?)

3. 평균을 내기 위해서 data type을 numeric 하게 바꾸어야 한다.

월 정보를 담고있는 row 때문에 column들을 일괄적으로 변경이 불가능

-> 년월일을 지정한 후 '월 정보를 담고있는 row' drop

이외의 오류

2012년도는 11월과 12월 데이터가 누락 되어 있다.

2013년도 데이터는 중복 되어있다.(같은 데이터 1년치가 한번 더 들어가있음)

2015년도 데이터는 10일이 누락 되어 있다.

년도별로 data type 형태와 표기가 다른 년도가 있다.

(columns 내의 각 row마다 데이터 타입이 다를 수도 있다) 잘 기억은 안남 ㅎㅎ

column 이름 인덱싱 잘 안됨 특히  format: day (공백이나 뭔가 있는 듯)

등등?